In [ ]:
### Last upd: 27.10.24

In [ ]:
# QN - вопросы
# rwrk - переделать/доработать

In [ ]:
import gc
gc.enable()

In [ ]:
# Cкорее всего полвина не нужна, но уже не могу вспомнть какие нужны, а какие нет

import os
import glob
import csv
import openpyxl

# import shutil
# import re
# import random

import math
import pandas as pd
import numpy as np

import datetime as dt
from datetime import date
from datetime import time

import matplotlib.pyplot as plt

import sklearn

import pickle

# from sklearn.utils import shuffle
# import mlxtend
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
# import scipy.stats as stats
# from sklearn import metrics
# from sklearn.metrics import roc_auc_score, f1_score, log_loss, classification_report, confusion_matrix, roc_curve, auc
# import functools
# from itertools import combinations
# from itertools import combinations_with_replacement
# from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.feature_selection import RFE
# from sklearn.feature_selection import RFECV
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import GridSearchCV
# import pandas.core.algorithms as algos
# from pandas import Series
# import traceback
# import string
# from ast import literal_eval
# from sklearn.model_selection import train_test_split
# import statsmodels.api as sm
# import statsmodels.formula.api as sm
# import pylab as pl
# import matplotlib.backends.backend_pdf
# from matplotlib.ticker import MaxNLocator
# import seaborn as sns
# from sklearn.pipeline import Pipeline
# import plotly.express as px

In [ ]:
# pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_rows', 20)

seed = 42

#### Фильтр предупреждений

In [ ]:
import warnings
# warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Функции (позже упаковать в библиотеки/пакеты)

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime as dt
from datetime import date
from datetime import time

## Разное

### Замена +/- бесокнечности

In [ ]:
# import math
# import pandas as pd
# import numpy as np


def inf_replacer(df,
                 x,
                 n_min_max):
    try:
        p_replace = np.nanmax(df[x][df[x] != np.inf])*n_min_max
        df[x] = df[x].replace(np.inf, p_replace)
    except:
        pass
    try:
        n_replace = np.nanmin(df[x][df[x] != -np.inf])*n_min_max
        df[x] = df[x].replace(-np.inf, n_replace)
    except:
        pass
        return df

## Функции для получения различных метрик

### PSI

In [ ]:
# import math
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt


class psi_():
    def fit(self,
            dev,
            val,
            feature,
            sample_wieght=None):
        self.feature = feature
        self.dev = dev
        self.val = val
        self.sample_wieght = sample_wieght  
        self.psi = pd.DataFrame()
        
        if sample_weight is not None:
            self.psi['dev_bin_size'] = self.dev.groupby('bin_index_'+self.feature)['sample_weight'].sum()
        else:
            self.psi['dev_bin_size'] = self.dev.groupby('bin_index_'+self.feature).size() / sum(self.dev.groupby('bin_index_'+self.feature).size())    
        
        if sample_weight is not None:
            self.psi['val_bin_size'] = self.val.groupby('bin_index_'+self.feature)['sample_weight'].sum() / sum(self.val.groupby('bin_index_'+self.feature)['sample_weight'].sum())
        else:
            self.psi['val_bin_size'] = self.val.groupby('bin_index_'+self.feature).size() / sum(self.val.groupby('bin_index_'+self.feature).size())
        
        # Для исключения деления на 0
        self.psi['dev_bin_size'] = np.where(self.psi['dev_bin_size']==0, 
                                              1e-6,
                                              self.psi['dev_bin_size'])
        self.psi['val_bin_size'] = np.where(self.psi['val_bin_size']==0, 
                                            1e-6, 
                                            self.psi['val_bin_size'])
        self.psi['dev_bin_size'] = self.psi['dev_bin_size'].fillna(1e-6)
        self.psi['val_bin_size'] = self.psi['val_bin_size'].fillna(1e-6)
        
        self.psi['PSI'] = (self.psi['dev_bin_size'] - self.psi['val_bin_size']) * (np.log(self.psi['dev_bin_size'] / self.psi['val_bin_size']))
        self.psi_val = self.psi['PSI'].sum()
        self.psi.loc['Total'] = self.psi.sum()
        self.psi['Feature'] = self.feature
        self.psi = self.psi[['Feature','dev_bin_size','val_bin_size','PSI']]
        self.psi.index.names = ['bin_index']
        
        return self
        
    def get_table(self):
        return self.psi
    def get_psi(self):
        return self.psi_val
    def plot(self,
             show=False,
             save = None):
        self.psi = self.psi.loc[self.psi.index!='Total']
        fig, ax = plt.subplots(figsize=(15, 8))  # Adjust the chart size here
        index = np.arange(len(self.psi))
        bar_width = 0.35
        dev_bars = ax.bar(index, 
                          self.psi['dev_bin_size'], 
                          bar_width, 
                          label='Development sample')
        val_bars = ax.bar(index + bar_width, 
                          self.psi['val_bin_size'], 
                          bar_width, 
                          label='Validation sample')
        ax.set_xlabel('Bins')
        ax.set_ylabel('Percentage')
        ax.set_title('WoE_'+self.feature+ f' Population Stability Index (PSI): {self.psi_val:.4f}')
        ax.set_xticks(index + bar_width / 2)
        ax.set_xticklabels(self.psi.index)
        ax.legend()
        fig.tight_layout()
        if save is not None:
            save_fig = os.path.join(save, 'PSI WoE_'+f'{self.feature}'+'.png')
            plt.savefig(save_fig)
        else:
            pass
        if show:
            plt.show()
        else:
            plt.close()
        return 

### Стабильность DR по бинам во времени (по годам) на основе изменения рангов

In [ ]:
# import math
# import pandas as pd
# import numpy as np
# import datetime as dt
# from datetime import date
# from datetime import time


def rank_stability_indexs(df,
                          feature,
                          freq ='y'): # 'y'/'yq'
    df['y'] = df['REP_DATE'].astype('datetime64[ns]').dt.year.astype('string') 
    df['yq'] = df['REP_DATE'].astype('datetime64[ns]').dt.year.astype('string')+'_Q'+df['REP_DATE'].astype('datetime64[ns]').dt.quarter.astype('string')
 
    dev_ranks = df.groupby('bin_index_'+feature)['target'].mean().rank().astype('Int64')
    dev_clean_ranks = df.loc[~df['bin_index_'+feature].isin(['Other','Missing','Special'])].groupby('bin_index_'+feature)['target'].mean().rank().astype('Int64')
    # Общая стабильность
    dr_stbl = pd.DataFrame()
    dr_stbl['bin_size'] = df.groupby(['bin_index_'+feature, freq])['target'].mean()
    dr_stbl = dr_stbl.unstack().rank().astype('Int64')

    dr_stbl_diff = dr_stbl.copy(deep=True)
    for i in dr_stbl.columns:
        dr_stbl_diff[i] = abs(dr_stbl_diff[i].sub(dev_ranks.to_numpy(), axis=0))
    rank_stability = round(1 - dr_stbl_diff.sum().sum()/dr_stbl_diff.size,2)

    clean_dr_stbl = dr_stbl.loc[~dr_stbl.index.isin(['Other','Missing','Special'])].rank().astype('Int64')
    clean_dr_stbl_diff = clean_dr_stbl.copy(deep=True)
    for i in clean_dr_stbl_diff.columns:
        clean_dr_stbl_diff[i] = abs(clean_dr_stbl_diff[i].sub(dev_clean_ranks.to_numpy(), axis=0))
    clean_rank_stability = round(1 - clean_dr_stbl_diff.sum().sum()/clean_dr_stbl_diff.size,2)
    rank_stab_tab = pd.DataFrame([[feature,
                                    rank_stability,
                                    clean_rank_stability]],
                                 columns = ('feature',
                                             'inner_rank_stability',
                                             'inner_clean_rank_stability')).set_index('feature')

    
    return rank_stab_tab

# Пример вызова функции
# rank_stab_tab = rank_stability_indexs(df, feature = 'bki_score')

### Ранговая корреляция Спирмена (сравнение разработки и OOT выборки)

In [ ]:
# import math
# import pandas as pd
# import numpy as np
# from scipy import stats


def spearmanr_stability(dev, 
                        val, 
                        feature,
                        target):
    dev_ranks = dev.groupby('bin_index_'+feature)[target].mean().rank()
    val_ranks = val.groupby('bin_index_'+feature)[target].mean().rank()
    
    dev_clean_ranks = dev.loc[~dev['bin_index_'+feature].isin(['Other','Missing','Special'])].groupby('bin_index_'+feature)[target].mean().rank()
    val_clean_ranks = val.loc[~val['bin_index_'+feature].isin(['Other','Missing','Special'])].groupby('bin_index_'+feature)[target].mean().rank()

    statistic, pvalue = stats.spearmanr(a=dev_clean_ranks, 
                                        b=val_clean_ranks, 
                                        axis=0, 
                                        nan_policy='propagate', 
                                        alternative='two-sided')
    
    statistic_clean, clean_pvalue = stats.spearmanr(a=dev_ranks, 
                                                  b=val_ranks, 
                                                  axis=0, 
                                                  nan_policy='propagate', 
                                                  alternative='two-sided')
    spearman_rs = pd.DataFrame([[feature,
                             round(statistic,5),
                             round(statistic_clean,5)]],
                           columns = ('feature',
                                      'Spearman_r_(dev/oot)',
                                      'clean_Spearman_r_(dev/oot)')).set_index('feature')
    return spearman_rs

## Функции биннинга

### Вспомогательные функции для биннинга

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import NearestNeighbors
from sklearn import tree
from sklearn.tree import _tree

from scipy import stats

sklearn.set_config(enable_metadata_routing=True)
seed = 42

#### Пре-процессинг для биннинга

In [ ]:
# Лист фреймов данных для биннинга
#------------------------------------------------------------------------------------ START
def pre_processing(df,
                   dtype , # 'categorical/numerical'
                   feature,
                   target,
                   sample_weight = None,
                   special_list = [],
                   cat_other_cutoff = None
                  ):
    other_list = []
    if (dtype == 'categorical') and (cat_other_cutoff is not None):
        others_to_missing, other_list = categorical_cutoff(df, 
                                                           feature, 
                                                           cat_other_cutoff,
                                                           sample_weight)
        if others_to_missing:
            df[feature] = np.where(df[feature].isin(other_list),
                                   np.nan,
                                   df[feature])
            other_list = []
    else:
        pass
## X, Y пропуски 
    x_na = df[[feature]].loc[df[feature].isna()]
    y_na = df[[target]][df.index.isin(x_na.index)]
## X, Y спец-значения 
    x_special = df[[feature]].loc[df[feature].isin(special_list)]
    y_special = df[[target]][df.index.isin(x_special.index)]
## X, Y менее cut_off % для категориальных показателей
    x_other = df[[feature]].loc[df[feature].isin(other_list)]
    y_other = df[[target]][df.index.isin(x_other.index)]
## X,Y clean
    x_clean = df[[feature]]\
    .loc[~df[feature].isna()]\
    .loc[~df[feature].isin(other_list)]\
    .loc[~df[feature].isin(special_list)]
    y_clean = df[[target]][df.index.isin(x_clean.index)]
## Sample_weight..  -_clean, -_other, -_special, -_na
    if sample_weight is not None:
        sw_clean = df[[sample_weight]][df.index.isin(x_clean.index)]
        sw_na = df[[sample_weight]][df.index.isin(x_na.index)]
        sw_special = df[[sample_weight]][df.index.isin(x_special.index)]
        sw_other = df[[sample_weight]][df.index.isin(x_other.index)]
    else:
        sw_clean = None
        sw_na = None
        sw_special = None
        sw_other = None
## frame_list[i] index для образения к объекту
    frame_list = [x_clean, # frame_list[0]
                  y_clean, # frame_list[1]
                  sw_clean, # frame_list[2]
                  x_na, # frame_list[3]
                  y_na, # frame_list[4] 
                  sw_na, # frame_list[5]
                  x_special, # frame_list[6] 
                  y_special, # frame_list[7] 
                  sw_special, # frame_list[8]
                  x_other, # frame_list[9]
                  y_other, # frame_list[10] 
                  sw_other] # frame_list[11]    

    return frame_list

#### Выбросы

In [ ]:
# Работа с выбросами (outlier_detector)
#------------------------------------------------------------------------------------ START
#------------------------------------------------------------------------------------ start(1)
## IQR detector
def iqr_filter(x):
    sub_x_clean = x.loc[
    np.array((x >= (stats.iqr(x)-1.5*x.min())))\
    & np.array((x <= (stats.iqr(x)+1.5*x.max())))
    ]
    return sub_x_clean
#------------------------------------------------------------------------------------ end(1)

#------------------------------------------------------------------------------------ start(2)
## KNN detector
def knn_filter(x,
               n_neighbors = 5):
    knn = NearestNeighbors(n_neighbors = n_neighbors).fit(x)
    mean_distances = pd.DataFrame(knn.kneighbors(x)[0]).mean(axis=1)
    threshold = mean_distances.quantile(0.95)
    clean_indxs = np.where(mean_distances <= threshold)
    sub_x_clean = x.iloc[clean_indxs]
    return sub_x_clean
#------------------------------------------------------------------------------------ end(2)

#------------------------------------------------------------------------------------ start(3)
##  Z_score_modified detector
def m_z_score_filter(x,
                     threshold=3.5):
##### Беру k=0,6745, применямый для полу-нормальных распределений, 
##### что позволит использовать MAD, как аналог стандартного отклонения
##### https://en.wikipedia.org/wiki/Median_absolute_deviation
    sub_x_clean =  x.loc[
    np.array((abs(0.6745 * (x - x.median()) / (np.median(np.abs(x - x.median()))))) < threshold)
    ]
    return sub_x_clean
#------------------------------------------------------------------------------------ end(3)

#------------------------------------------------------------------------------------ start(4)
## Z_score detector
def z_score_filter(x,
                   threshold=3):
    sub_x_clean = x.loc[np.array(stats.zscore(x, ddof = 1)) < threshold]
    return sub_x_clean
#------------------------------------------------------------------------------------ END

#### Проверка направления тренда 

In [ ]:
# Проверка направления связи
#------------------------------------------------------------------------------------ START
def trend_check(x,
                y, 
                sample_weight=None,
                n_bins=20):
    if sample_weight is not None:
        sample_weight = sample_weight
    else:
        sample_weight = None
## Дискретизация данных
    kbd = KBinsDiscretizer(n_bins = n_bins,
                           strategy='quantile', # uniform
                           random_state = seed,
                           subsample = None,
                           encode = 'ordinal'
                          )
    kbd.fit(x, 
            sample_weight = sample_weight).set_fit_request(sample_weight=True)    
    mono_check = pd.DataFrame({'x':pd.Series(x.ravel()),
                               'target':pd.Series(y),
                               'sample_weight':pd.Series(sample_weight)},
                             )
    mono_check['bin_index'] = kbd.transform(x).astype('int64')
    mono_check_ = pd.DataFrame()
    if sample_weight is not None:
        mono_check_['sample_weight'] = mono_check.groupby('bin_index')['sample_weight'].sum()
    else:
        pass
    mono_check_['target'] = mono_check.groupby('bin_index')['target'].mean()
    X = mono_check_.index.values
    Y = mono_check_['target'].values
    if sample_weight is not None:
        sample_weight = mono_check_['sample_weight'].values
    else:
        sample_weight = None
### Полином 1 степени (линейная регрессия) для определения наклона лнии тренда
    lr_coef = np.polyfit(X, 
                         Y,
                         w=sample_weight,
                         deg=1)[0]
### Определение monotonic_cst исходя из коэффициента наклона уравнения линейной регрессии
    if lr_coef > 0:
        monotonic_cst = 1 # Increase
        lr_sense = 'Increase'
    elif lr_coef < 0:
        monotonic_cst = -1 # Decrease
        lr_sense = 'Decrease'
    elif lr_coef == 0:
        monotonic_cst = 0 # None
        lr_sense = 'None'

    return lr_sense, monotonic_cst
#------------------------------------------------------------------------------------ END

#### "Others" - выделение редких наблюдений

In [ ]:
# Выделение редко-встречающихся (менее, чем в x% случаев) категорий в отдельный бин
#------------------------------------------------------------------------------------ START
def categorical_cutoff(df, 
                       feature,
                       cat_other_cutoff=None, 
                       sample_weight=None):
    if sample_weight is None:
        x = df[feature]
        cutoff_count = np.ceil(cat_other_cutoff * len(x))
        cat_tab = pd.Series(x).value_counts()
        other_list = cat_tab[cat_tab < cutoff_count].index.values
        mask_others = pd.Series(x).isin(other_list).values
        min_other_size = np.ceil(len(x)*0.01)
        other_size = len(x[x.isin(other_list)])
        others_to_missing = min_other_size >= other_size
    else:
        x = df[[feature,sample_weight]]
        cutoff_weight = cat_other_cutoff * sum(x[sample_weight])
        # pd.concat([x,df[sample_weight]], ignore_index=False, axis = 1)
        cat_tab = x.groupby([feature])[sample_weight].agg('sum')
        other_list = cat_tab[cat_tab < cutoff_weight].index.values
        mask_others = pd.Series(x[feature]).isin(other_list).values
        min_other_size = x[sample_weight].sum()*0.01
        other_size = x[x.isin(other_list)][sample_weight].sum()
        others_to_missing = min_other_size >= other_size 
    if np.count_nonzero(~mask_others) == 0:
            raise ValueError(str('Все значения ' + feature + ' попали в категорию Others'))
    return others_to_missing, other_list

# # Пример вызова функции 
# other_list= categorical_cutoff(df = df_cat, 
#                                 feature='reg_region', 
#                                 cat_other_cutoff = 0.01,
#                                 sample_weight = 'sample_weight')
#------------------------------------------------------------------------------------ END

### Биннинг

#### Биннинг категориальных (дискретных) показателей

In [ ]:
#------------------------------------------------------------------------------------ START
class categorical_binning:
#------------------------------------------------------------------------------------ start(1)
## Инициация биннинга, задать парамтеры для фунции
    def __init__(self,
                 target_type, # 'binary'/'continuous'
                 n_bins, # Количество бакетов в биннинге
                 seed,
                 min_bin_size, # Минимальный размер бакета,
                 cat_other_cutoff = None,
                 woe_calc_method = None # 'sum_surrogate'/'mean_surrogate'
                 ):
        self.seed = seed
        self.target_type = target_type
        self.n_bins = n_bins
        self.min_bin_size = min_bin_size
        self.cat_other_cutoff = cat_other_cutoff
        self.woe_calc_method = woe_calc_method  
#------------------------------------------------------------------------------------ end(1)

#------------------------------------------------------------------------------------ start(2)
## К инициированному объекту применяется .fit() с заданными параметрами
    def fit(self, 
            df,
            feature,
            target,
            special_list =[],
            sample_weight = None,
            dtype = 'categorical'):
### Подготовка вводных данных для обучения функции биннинга  
        self.frame_list = pre_processing(df,
                                         dtype,
                                         feature,
                                         target,
                                         sample_weight,
                                         special_list,
                                         self.cat_other_cutoff)
        self.feature = feature
        self.sample_weight = sample_weight
        self.target = target
        #------------------------------------------------------------
        # Индексы объектов в frame_list
        # x_clean = frame_list[0]
        # y_clean = frame_list[1]
        # sw_clean = frame_list[2]
        # x_na = frame_list[3]
        # y_na = frame_list[4] 
        # sw_na = frame_list[5]
        # x_special = frame_list[6] 
        # y_special = frame_list[7] 
        # sw_special = frame_list[8]
        # x_other = frame_list[9]
        # y_other = frame_list[10] 
        # sw_other = frame_list[11]
        #------------------------------------------------------------

#### Alphabetical encoding
        # encoder = preprocessing.LabelEncoder()
        # encoder.fit(self.frame_list[0].values.ravel())
        # X = encoder.transform(self.frame_list[0].values.ravel()).reshape(-1, 1)
####----------------------------------------------------------------- encoding end
#### Target mean encoding
        encoder = preprocessing.TargetEncoder(random_state = self.seed,
                                              target_type = self.target_type)
        encoder.fit(self.frame_list[0].values.reshape(-1, 1),
                    self.frame_list[1].values.ravel())
        X = encoder.transform(self.frame_list[0].values.reshape(-1, 1))
        encoding_map = dict(zip(X.ravel(),
                                self.frame_list[0].values.ravel()
                               )
                           )
####----------------------------------------------------------------- encoding end
        
        Y = self.frame_list[1].values.ravel()
        if self.sample_weight is not None:
            S_W = self.frame_list[2].values.ravel()
        else:
            S_W = None
### Бинарный таргет target_type == 'binary'
### CART - Decision Tree Classifier 
        if self.target_type == 'binary':
            cart_kwargs = {'min_samples_leaf' : self.min_bin_size,
                           'max_leaf_nodes' : self.n_bins,
                           'random_state' : self.seed,
                           'criterion' : 'gini'}    
            cart_model = DecisionTreeClassifier(**cart_kwargs)
            fit_args = {'X':X,
                        'y':Y,
                        'sample_weight':S_W}
            cart_model.fit(**fit_args)     
### Бинарный таргет target_type == 'continuous'
### CART - Decision Tree Regressor
        elif self.target_type == 'continuous':
            cart_kwargs = {'min_samples_leaf' : self.min_bin_size,
                           'max_leaf_nodes' : self.n_bins,
                           'random_state' : self.seed,
#### MAE более устойчив к выбросам, чем MSE, поэтому для качественных признаков выбран MSE, а для количественных MAE
                           'criterion' : 'friedman_mse'}
            cart_model = DecisionTreeRegressor(**cart_kwargs)
            fit_args = {'X':X,
                        'y':Y,
                        'sample_weight':S_W}
            cart_model.fit(**fit_args)
        # return self
### Построение внутренней таблицы - self.tab для расчета woe
#### Основная таблица tab

#### Alphabetical reverse encoding
        # data = {self.feature : encoder.inverse_transform(X.ravel())}
####----------------------------------------------------------------- reverse encoding end
#### Target mean reverse encoding
        data = {self.feature : pd.Series(X.ravel()).map(encoding_map)}
####----------------------------------------------------------------- reverse encoding end
        data.update({'bin_index' : cart_model.apply(X, check_input=True),
                     'target' : self.frame_list[1].values.ravel()})
        if self.sample_weight is not None:
            data.update({'sample_weight' : self.frame_list[2].values.ravel()})
        else:
            pass
        self.tab = pd.DataFrame(data)       
        if self.sample_weight is not None:
            index_map = (self.tab.loc[self.tab['target']==1].groupby('bin_index')['sample_weight'].sum()/ \
            self.tab.groupby('bin_index')['sample_weight'].sum()).reset_index()
            index_map['bin_true'] = stats.rankdata(index_map['sample_weight']).astype('int64')
            index_map = dict(zip(index_map['bin_index'],
                                 index_map['bin_true']))
            self.tab['bin_index'] = self.tab['bin_index'].map(index_map)
        else:
            index_map = (self.tab.loc[self.tab['target']==1].groupby('bin_index').size()/ \
            self.tab.groupby('bin_index').size()).reset_index()
            index_map['bin_true'] = stats.rankdata(index_map[0]).astype('int64')
            index_map = dict(zip(index_map['bin_index'],
                                 index_map['bin_true']))
            self.tab['bin_index'] = self.tab['bin_index'].map(index_map)
#### Дополнительные таблицы для tab 
##### Таблица с пропусками
        data = {self.feature:self.frame_list[3].values.ravel(),
                'bin_index':'Missing',
                'target':self.frame_list[4].values.ravel()}
        if self.sample_weight is not None:
            data.update({'sample_weight':self.frame_list[5].values.ravel()})
        else:
            pass
        na_tab = pd.DataFrame(data)
##### Таблица с специальными значениями
        data = {self.feature:self.frame_list[6].values.ravel(),
                'bin_index':'Special',
                'target':self.frame_list[7].values.ravel()}
        if self.sample_weight is not None:
            data.update({'sample_weight':self.frame_list[8].values.ravel()})
        else:
            pass
        special_tab = pd.DataFrame(data)
##### Таблица с Прочими (менее i%)
        data = {self.feature:self.frame_list[9].values.ravel(),
                'bin_index':'Other',
                'target':self.frame_list[10].values.ravel()}
        if self.sample_weight is not None:
            data.update({'sample_weight':self.frame_list[11].values.ravel()})
        else:
            pass
        other_tab = pd.DataFrame(data)
#### Создание общей таблицы с дополнительными значениями
        tab_add = pd.concat([na_tab,
                             special_tab,
                             other_tab]).reset_index(drop=True)

##### Снижение размера выходных pickle-файлов        
        del na_tab, special_tab, other_tab
#### Индексы для таблицы WoE       
        self.indxs = ['Other','Special','Missing']
        self.indxs = sorted((set(self.indxs))&\
                       set(list(tab_add['bin_index'].unique())), key = self.indxs.index)
        self.indxs = list(np.sort(self.tab['bin_index'].unique()))+self.indxs   
        self.tab = pd.concat([self.tab,
                              tab_add]).reset_index(drop=True)
        del tab_add
### Создание объекта splits - словаря полученных в результате автоматического разбиения классов, для трансформации
        self.user_map = None
        self.splits = {}
        self.splits = self.tab.groupby('bin_index')[self.feature].unique().to_dict()
        self.splits = dict(zip(
            self.splits.keys(),
            map(tuple,self.splits.values())
        ))
        self.auto_map = {}
        for key, values in self.splits.items():
            for item in values:
                self.auto_map[item] = key    
        return self
#------------------------------------------------------------------------------------ end(2)

#------------------------------------------------------------------------------------ start(3)
## Создание объекта splits - словаря полученных в результате автоматического разбиения классов
#### Можно сделать через @property, но работает и так
    def get_auto_splits(self):
        return self.splits
#------------------------------------------------------------------------------------ end(3)

#------------------------------------------------------------------------------------ start(4)
## Пользовательская (ручная) коррекция полученного разбиения на классы
    def user_splits(self,
                    user_splits=None):
        if user_splits is not None:
#### user_splits при необходимости коррекции разбиения задается, как dict-like объект
#### user_splits = {1:('a','b','c'), 2:('d','e','f'),...,'Special':('...'),...}
            self.user_map = {}
            for key, values in user_splits.items():
                for item in values:
                    self.user_map[item] = key
            self.tab['bin_index'] = self.tab[self.feature].map(self.user_map)
        else:
            pass
        return self
#------------------------------------------------------------------------------------ end(4)
    
#------------------------------------------------------------------------------------ start(5)
## Расчет WoE, IV и получение итоговых таблиц tab и woe
    def calculate_woe(self):
        self.woe = pd.DataFrame()
### Расчет для бинарных показателей
        if self.target_type == 'binary':
            if self.sample_weight is not None:
                self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                self.woe['event'] = self.tab.loc[self.tab['target']==1].groupby('bin_index')['sample_weight'].sum()
            else:
                self.woe['bin_size'] = self.tab.groupby('bin_index').size()
                self.woe['event'] = self.tab.loc[self.tab['target']==1].groupby('bin_index').size()
            self.woe['event'] = self.woe['event'].fillna(1e-6) # ВОЗМОЖНОЕ МЕСТО ОШИБКИ
            self.woe['non_event'] = self.woe['bin_size'] - self.woe['event']
            self.woe['event_rate'] = self.woe['event'] / self.woe['bin_size'] 
            self.woe['dstr_event'] = self.woe['event'] / sum(self.woe['event'])
            self.woe['dstr_non_event'] = self.woe['non_event'] / sum(self.woe['non_event'])
            self.woe['WoE'] = np.log(self.woe['dstr_non_event'] / self.woe['dstr_event'])
            self.woe['IV'] = sum(abs((self.woe['dstr_non_event'] - self.woe['dstr_event']) * self.woe['WoE']))
            self.woe['feature'] = self.feature
            self.woe['values_in_bin'] = self.tab.groupby('bin_index')[self.feature].unique()
            self.woe = self.woe[['feature',
                       'bin_size',
                       'event',
                       'event_rate',
                       'WoE',
                       'IV',
                       'values_in_bin']]
            self.woe = self.woe.reindex(self.indxs)
### Расчет для непрерывных показателей
        elif self.target_type == 'continuous':
##### QN - Взвешивание целевой переменной (не уверен надо ли это)
            if self.sample_weight is not None:
                self.tab['target_weighted']=self.tab['target']*self.tab['sample_weight']  
#### 'sum_surrogate'/'mean_surrogate'
            if self.woe_calc_method == 'mean_surrogate':
### Описание: https://gnpalencia.org/optbinning/tutorials/tutorial_continuous.html
                self.woe = pd.DataFrame()
                if self.sample_weight is not None:
                    self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                    self.woe['bin_mean'] = self.tab.groupby('bin_index')['target_weighted'].mean()
                    self.woe['mean_target'] = self.tab['target_weighted'].mean()
                else:
                    self.woe['bin_size'] = self.tab.groupby('bin_index').size()
                    self.woe['bin_mean'] = self.tab.groupby('bin_index')['target'].mean()
                    self.woe['mean_target'] = self.tab['target'].mean()
                self.woe['dstr_obs'] = self.woe['bin_size'] / self.woe['bin_size'].sum()
                self.woe['WoE'] = self.woe['bin_mean'] - self.woe['mean_target']
                self.woe['IV'] = sum(self.woe['dstr_obs']*abs(self.woe['WoE']))
                self.woe['feature'] = self.feature
                self.woe['values_in_bin'] = self.tab.groupby('bin_index')[self.feature].unique()
                self.woe = self.woe[['feature',
                           'bin_size',
                           'bin_mean',
                           'mean_target',
                           'WoE',
                           'IV',
                           'values_in_bin']]
                self.woe = self.woe.reindex(self.indxs)
            elif self.woe_calc_method == 'sum_surrogate':
#### Опсиание: https://www.listendata.com/2019/08/WOE-IV-Continuous-Dependent.html
                self.woe = pd.DataFrame()
                if self.sample_weight is not None:
                    self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                    self.woe['target_sum'] = self.tab.groupby('bin_index')['target_weighted'].sum()
                else:
                    self.woe['bin_size'] = self.tab.groupby('bin_index').size()    
                    self.woe['target_sum'] = self.tab.groupby('bin_index')['target'].sum()
                self.woe['dstr_obs'] = self.woe['bin_size'] / self.woe['bin_size'].sum()
                self.woe['dstr_target'] = self.woe['target_sum'] / self.woe['target_sum'].sum()  
                self.woe['WoE'] = np.log(self.woe['dstr_target'] / self.woe['dstr_obs'])
                self.woe['IV'] = abs((( self.woe['dstr_target'] - self.woe['dstr_obs']) * self.woe['WoE'])).sum()
                self.woe['feature'] = self.feature
                self.woe['values_in_bin'] = self.tab.groupby('bin_index')[self.feature].unique()
                self.woe = self.woe[['feature',
                           'bin_size',
                           'target_sum',
                           'WoE',
                           'IV',
                           'values_in_bin']]
                self.woe = self.woe.reindex(self.indxs)
            elif self.woe_calc_method is None:
                print("Задайте параметр woe_calc_method во время инициации функции ('sum_surrogate'/'mean_surrogate')")
        self.woe['WoE'] = np.where(self.woe['event']==1e-6, 0, self.woe['WoE']) # ЕЩЕ ОДНО МЕСТО ВОЗМОЖНОЙ ОШИБКИ
        self.woe['WoE'] = np.where(self.woe['event']==0, 0, self.woe['WoE']) # ЕЩЕ ОДНО МЕСТО ВОЗМОЖНОЙ ОШИБКИ
##### Снижение размера выходных pickle-файлов
        del self.tab
        del self.frame_list
        return self
## Извлечение таблицы WoE
    def get_woe_table(self):
        return self.woe
## WoE трансформация показателей в исходном DataFrame()
    def woe_transform(self,
                      df,
                      drop_bin_index = False): # Заполнить новые значения 'Other' или 'Missing'
        if self.user_map is not None:
            df['bin_index_'+self.feature] = df[self.feature].map(self.user_map)
        else:
            df['bin_index_'+self.feature] = df[self.feature].map(self.auto_map)
            
        if 'Other' not in self.indxs:
            df['bin_index_'+self.feature] = df['bin_index_'+self.feature].fillna('Missing')
        else:
            df['bin_index_'+self.feature] = df['bin_index_'+self.feature].fillna('Other')

        df['WoE_'+self.feature]=df['bin_index_'+self.feature].map((self.woe[['WoE']].T.to_dict('records'))[0])
        if drop_bin_index:
            df = df.drop(columns = ['bin_index_'+self.feature], errors='ignore')
        else:
            pass
        return df
#------------------------------------------------------------------------------------ end(5)
#------------------------------------------------------------------------------------ END

#### Биннинг числовых показателей

In [ ]:
#------------------------------------------------------------------------------------ START
class numerical_binning:
#------------------------------------------------------------------------------------ start(1)
## Инициация биннинга, задать парамтеры для фунции
    def __init__(self,
                 target_type, # 'binary'/'continuous'
                 n_bins, # Количество бакетов в биннинге
                 seed,
                 min_bin_size, # Минимальный размер бакета,
                 outlier_detector = None, # 'iqr','knn','z_score','z_score_mod'
                 n_neighbors = 5, # если выбран метод outlier_detector == 'knn' можно задать "количество ближайших соседей"
                 woe_calc_method = None # 'sum_surrogate'/'mean_surrogate'
                 ):
        self.seed = seed
        self.target_type = target_type
        self.n_bins = n_bins
        self.min_bin_size = min_bin_size
        self.woe_calc_method = woe_calc_method 
        self.outlier_detector = outlier_detector
        if self.outlier_detector == 'knn':
            self.n_neighbors = n_neighbors
        else:
            pass
#------------------------------------------------------------------------------------ end(1)

#------------------------------------------------------------------------------------ start(2)
## К инициированному объекту применяется .fit() с заданными параметрами
    def fit(self, 
            df,
            feature,
            target,
            special_list =[],
            sample_weight = None,
            dtype = 'numerical'):
### Подготовка вводных данных для обучения функции биннинга  
        self.frame_list = pre_processing(df,
                                         dtype,
                                         feature,
                                         target,
                                         sample_weight,
                                         special_list)
### Условный тест на дискретность с наложенным ограничением на количество бинов
#### Лучше дискретные показатели проводить через категориальный биннинг
        if self.frame_list[0].nunique().iloc[0] < self.n_bins:
            self.n_bins = self.frame_list[0].nunique().iloc[0]
            is_discrete = True
        else:
            is_discrete = False
        if is_discrete:
            self.outlier_detector = None
        else:
            pass
        self.special_list = special_list
        self.feature = feature
        self.sample_weight = sample_weight
        self.target = target
        #------------------------------------------------------------
        # Индексы объектов в frame_list
        # x_clean = frame_list[0]
        # y_clean = frame_list[1]
        # sw_clean = frame_list[2]
        # x_na = frame_list[3]
        # y_na = frame_list[4] 
        # sw_na = frame_list[5]
        # x_special = frame_list[6] 
        # y_special = frame_list[7] 
        # sw_special = frame_list[8]
        # x_other = frame_list[9]
        # y_other = frame_list[10] 
        # sw_other = frame_list[11]
        #------------------------------------------------------------
        if self.outlier_detector is not None:
            if self.outlier_detector == 'iqr':
                X = iqr_filter(self.frame_list[0])
            elif self.outlier_detector == 'knn':
                 X = knn_filter(self.frame_list[0], 
                                n_neighbors = self.n_neighbors)
            elif self.outlier_detector == 'z_score':
                X = z_score_filter(self.frame_list[0])
            elif self.outlier_detector =='z_score_mod':
                X = m_z_score_filter(self.frame_list[0])        
            Y = self.frame_list[1][self.frame_list[1].index.isin(X.index)].values.ravel()
            if self.sample_weight is not None:
                S_W = self.frame_list[2][self.frame_list[2].index.isin(X.index)].values.ravel()
            else:
                S_W = None
            X = X.values.ravel().reshape(-1, 1)
        else:
            X = self.frame_list[0].values.ravel().reshape(-1, 1)
            Y = self.frame_list[1].values.ravel()
            if self.sample_weight is not None:
                S_W = self.frame_list[2].values.ravel()
            else:
                S_W = None
        if is_discrete:
            monotonic_cst = 0
            self.lr_sense = 'None (discrete feature)'
        else:
            self.lr_sense, monotonic_cst = trend_check(x=X,
                                                       y=Y, 
                                                       sample_weight = S_W)
### Бинарный таргет target_type == 'binary'
### CART - Decision Tree Classifier 
        if self.target_type == 'binary':
            cart_kwargs = {'min_samples_leaf' : self.min_bin_size,
                           'max_leaf_nodes' : self.n_bins,
                           'random_state' : self.seed,
                           'monotonic_cst': [monotonic_cst],
                           'criterion' : 'gini'}    
            cart_model = DecisionTreeClassifier(**cart_kwargs)
            fit_args = {'X':X,
                        'y':Y,
                        'sample_weight':S_W}
            cart_model.fit(**fit_args)     
### Непрерывный таргет target_type == 'continuous'
### CART - Decision Tree Regressor
        elif self.target_type == 'continuous':
            cart_kwargs = {'min_samples_leaf' : self.min_bin_size,
                           'max_leaf_nodes' : self.n_bins,
                           'random_state' : self.seed,
                           'monotonic_cst': [monotonic_cst],
#### MAE более устойчив к выбросам, чем MSE, поэтому для качественных признаков выбран MSE, а для количественных MAE
                           'criterion' : 'absolute_error'}
            cart_model = DecisionTreeRegressor(**cart_kwargs)
            fit_args = {'X':X,
                        'y':Y,
                        'sample_weight':S_W}
            cart_model.fit(**fit_args)
### Получение split_points
        self.split_points = cart_model.tree_.threshold[cart_model.tree_.threshold!=_tree.TREE_UNDEFINED]
        self.split_points = np.append(self.split_points, 
                                      np.inf) # Верхние границы (включительно)
        self.split_points.sort()
### Построение внутренней таблицы - self.tab для расчета woe
#### Основная таблица tab_base       
        bin_indexs = pd.cut(self.frame_list[0].values.ravel(),
                            bins = np.append(-np.inf, 
                                             self.split_points),
                            include_lowest=True,
                            labels=False)+1
        data = {feature : self.frame_list[0].values.ravel(), 
                'bin_index' : bin_indexs, 
                'target' : self.frame_list[1].values.ravel()} 
        if self.sample_weight is not None:
            data.update({'sample_weight' : self.frame_list[2].values.ravel()})
        else:
            pass
        self.tab_base = pd.DataFrame(data)
        del data
        self.tab_base['bin_index'] = self.tab_base['bin_index'].astype('Int64')
        
#### Дополнительные таблицы для tab 
##### Таблица с пропусками
        data = {self.feature:self.frame_list[3].values.ravel(),
                'bin_index':'Missing',
                'target':self.frame_list[4].values.ravel()}
        if self.sample_weight is not None:
            data.update({'sample_weight':self.frame_list[5].values.ravel()})
        else:
            pass
        na_tab = pd.DataFrame(data)
##### Таблица со специальными значениями
        data = {self.feature:self.frame_list[6].values.ravel(),
                'bin_index':'Special',
                'target':self.frame_list[7].values.ravel()}
        if self.sample_weight is not None:
            data.update({'sample_weight':self.frame_list[8].values.ravel()})
        else:
            pass
        special_tab = pd.DataFrame(data)
        del data
#### Создание общей таблицы с дополнительными значениями
        self.tab_add = pd.concat([na_tab,
                                  special_tab]).reset_index(drop=True)

##### Снижение размера выходных pickle-файлов
        del na_tab, special_tab

#### Индексы для таблицы WoE       
        self.indxs = ['Special','Missing']
        self.indxs = sorted((set(self.indxs))&\
                            set(list(self.tab_add['bin_index'].unique())), key = self.indxs.index)
        self.indxs = list(np.sort(self.tab_base['bin_index'].unique()))+self.indxs
#### Итоговая таблица self.tab
        self.tab = pd.concat([self.tab_base,
                              self.tab_add]).reset_index(drop=True)
### Создание объекта splits - словаря полученных в результате автоматического разбиения классов, для трансформации
        return self
#------------------------------------------------------------------------------------ end(2)

#------------------------------------------------------------------------------------ start(3)
#### Можно сделать через @property, но работает и так
    def get_auto_split_points(self):
        return self.split_points
#------------------------------------------------------------------------------------ end(3)

#------------------------------------------------------------------------------------ start(4)
## Пользовательская (ручная) коррекция полученных границ бинов 
#### Применяется только к основным данным
    def user_split_points(self,
                          user_split_points=None):
#### user_splits при необходимости коррекции разбиения задается, как array-like объект из верхних границ (n_bins-1) бинов 
#### user_splits = np.array([604.5, 651.5, 670.5, inf]) ! по возрастанию
        if user_split_points is not None:
            bin_indexs = pd.cut(self.frame_list[0].values.ravel(),
                                bins = np.append(-np.inf, 
                                                 user_split_points),
                                include_lowest=True,
                                labels=False)+1
            data = {feature : self.frame_list[0].values.ravel(), 
                    'bin_index' : bin_indexs, 
                    'target' : self.frame_list[1].values.ravel()}
            self.tab_base = pd.DataFrame(data)
            self.tab_base['bin_index'] = self.tab_base['bin_index'].astype('Int64')
            if self.sample_weight is not None:
                data.update({'sample_weight' : self.frame_list[2].values.ravel()})
            else:
                pass
            self.tab_base = pd.DataFrame(data)
            self.tab = pd.concat([self.tab_base,
                                  self.tab_add]).reset_index(drop=True)
        else:
            pass
        return self
#------------------------------------------------------------------------------------ end(4)

#------------------------------------------------------------------------------------ start(5)
## Расчет WoE, IV и получение итоговых таблиц tab и woe
    def calculate_woe(self):
        self.woe = pd.DataFrame()
### Расчет для бинарных показателей
        if self.target_type == 'binary':
            if self.sample_weight is not None:
                self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                self.woe['event'] = self.tab.loc[self.tab['target']==1].groupby('bin_index')['sample_weight'].sum()
            else:
                self.woe['bin_size'] = self.tab.groupby('bin_index').size()
                self.woe['event'] = self.tab.loc[self.tab['target']==1].groupby('bin_index').size() 

            self.woe['event'] = self.woe['event'].fillna(1e-6) # МЕСТО ВОЗМОЖНОЙ ОШИБКИ
            self.woe['non_event'] = self.woe['bin_size'] - self.woe['event']
            self.woe['event_rate'] = self.woe['event'] / self.woe['bin_size'] 
            self.woe['dstr_event'] = self.woe['event'] / sum(self.woe['event'])
            self.woe['dstr_non_event'] = self.woe['non_event'] / sum(self.woe['non_event'])
            self.woe['WoE'] = np.log(self.woe['dstr_non_event'] / self.woe['dstr_event'])
            self.woe['IV'] = sum(abs((self.woe['dstr_non_event'] - self.woe['dstr_event']) * self.woe['WoE']))
            self.woe['feature'] = self.feature
            self.woe['upper_bound'] = self.tab.groupby('bin_index')[self.feature].max()
            self.woe['upper_bound'] = self.woe['upper_bound'].replace(self.woe['upper_bound'].max(), np.inf)
            self.woe['trend'] = self.lr_sense
            self.woe = self.woe[['feature',
                                 'bin_size',
                                 'event',
                                 'event_rate',
                                 'WoE',
                                 'IV',
                                 'upper_bound',
                                 'trend']]
            self.woe = self.woe.reindex(self.indxs)
### Расчет для непрерывных показателей
        elif self.target_type == 'continuous':
##### QN - Взвешивание целевой переменной (не уверен надо ли это)
            # if self.sample_weight is not None:
            #     self.tab['target_weighted']=self.tab['target']*self.tab['sample_weight'] 
#### 'sum_surrogate'/'mean_surrogate'
            if self.woe_calc_method == 'mean_surrogate':
### Описание: https://gnpalencia.org/optbinning/tutorials/tutorial_continuous.html
                self.woe = pd.DataFrame()
                if self.sample_weight is not None:
                    self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                    self.woe['bin_mean'] = self.tab.groupby('bin_index')['target'].mean()
                    # self.woe['bin_mean'] = self.tab.groupby('bin_index')['target_weighted'].mean()
                    self.woe['mean_target'] = self.tab['target'].mean()
                    # self.woe['mean_target'] = self.tab['target_weighted'].mean()
                else:
                    self.woe['bin_size'] = self.tab.groupby('bin_index').size()
                    self.woe['bin_mean'] = self.tab.groupby('bin_index')['target'].mean()
                    self.woe['mean_target'] = self.tab['target'].mean()
                self.woe['dstr_obs'] = self.woe['bin_size'] / self.woe['bin_size'].sum()
                self.woe['WoE'] = self.woe['bin_mean'] - self.woe['mean_target']
                self.woe['IV'] = sum(self.woe['dstr_obs']*abs(self.woe['WoE']))
                self.woe['feature'] = self.feature
                self.woe['upper_bound'] = self.tab.groupby('bin_index')[self.feature].max()
                self.woe['upper_bound'] = self.woe['upper_bound'].replace(self.woe['upper_bound'].max(), np.inf)
                self.woe['trend'] = self.lr_sense
                self.woe = self.woe[['feature',
                                     'bin_size',
                                     'bin_mean',
                                     'mean_target',
                                     'WoE',
                                     'IV',
                                     'upper_bound',
                                     'trend']]
                self.woe = self.woe.reindex(self.indxs)
            elif self.woe_calc_method == 'sum_surrogate':
#### Опсиание: https://www.listendata.com/2019/08/WOE-IV-Continuous-Dependent.html
                self.woe = pd.DataFrame()
                if self.sample_weight is not None:
                    self.woe['bin_size'] = self.tab.groupby('bin_index')['sample_weight'].sum()
                    self.woe['target_sum'] = self.tab.groupby('bin_index')['target'].sum()
                    # self.woe['target_sum'] = self.tab.groupby('bin_index')['target_weighted'].sum()
                else:
                    self.woe['bin_size'] = self.tab.groupby('bin_index').size()    
                    self.woe['target_sum'] = self.tab.groupby('bin_index')['target'].sum()
                self.woe['dstr_obs'] = self.woe['bin_size'] / self.woe['bin_size'].sum()
                self.woe['dstr_target'] = self.woe['target_sum'] / self.woe['target_sum'].sum()  
                self.woe['WoE'] = np.log(self.woe['dstr_target'] / self.woe['dstr_obs'])
                self.woe['IV'] = abs((( self.woe['dstr_target'] - self.woe['dstr_obs']) * self.woe['WoE'])).sum()
                self.woe['feature'] = self.feature
                self.woe['upper_bound'] = self.tab.groupby('bin_index')[self.feature].max()
                self.woe['upper_bound'] = self.woe['upper_bound'].replace(self.woe['upper_bound'].max(), np.inf)
                self.woe['trend'] = self.lr_sense
                self.woe = self.woe[['feature',
                                     'bin_size',
                                     'target_sum',
                                     'WoE',
                                     'IV',
                                     'upper_bound',
                                     'trend']]
                self.woe = self.woe.reindex(self.indxs)
            elif self.woe_calc_method is None:
                print("Задайте параметр woe_calc_method во время инициации функции ('sum_surrogate'/'mean_surrogate')")
        self.woe['WoE'] = np.where(self.woe['event']==1e-6, 0, self.woe['WoE']) # ЕЩЕ ОДНО МЕСТО ВОЗМОЖНОЙ ОШИБКИ
        self.woe['WoE'] = np.where(self.woe['event']==0, 0, self.woe['WoE']) # ЕЩЕ ОДНО МЕСТО ВОЗМОЖНОЙ ОШИБКИ
##### Снижение размера выходных pickle-файлов
        del self.frame_list
        del self.tab
        del self.tab_base
        del self.tab_add
        return self
## Извлечение таблицы WoE
    def get_woe_table(self):
        return self.woe
## WoE трансформация показателей в исходном DataFrame()
    def woe_transform(self,
                      df,
                      drop_bin_index = False):
        # regular_bounds = np.array(self.woe[['upper_bound']].loc[~self.woe.index.isin(['Other',
        #                                                                               'Special',
        #                                                                               'Missing'])])
        # if any (regular_bounds<=0) is True:
        #     regular_bounds = np.append(-np.inf,regular_bounds) 
        # else:
        #     regular_bounds = np.append(0,regular_bounds)        
        regular_bounds = np.append(-np.inf,
                                   np.array(self.woe[['upper_bound']].loc[~self.woe.index.isin(['Other',
                                                                                                'Special',
                                                                                                'Missing'])]))
        
        regular_bounds.sort()
        df['bin_index_'+self.feature] = pd.cut(df[self.feature].values.ravel(),
                                               bins = regular_bounds,
                                               include_lowest=True,
                                               labels=False)+1
        df['bin_index_'+self.feature] = df['bin_index_'+self.feature].astype('Int64').astype('string')
        df['bin_index_'+self.feature] = df['bin_index_'+self.feature].fillna('Missing')
        df['bin_index_'+self.feature] = np.where(df[self.feature].isin(self.special_list),
                                                 'Special', 
                                                 df['bin_index_'+self.feature])
        self.woe.index = self.woe.index.astype('string')
        df['WoE_'+self.feature]=df['bin_index_'+self.feature].map((self.woe[['WoE']].T.to_dict('records'))[0])
        if drop_bin_index:
            df = df.drop(columns = ['bin_index_'+self.feature], errors='ignore')
        else:
            pass
        return df
#------------------------------------------------------------------------------------ end(5)
#------------------------------------------------------------------------------------ END

# Вводные модели

## Идентификаторы модели и наблюдений

In [ ]:
reg_name = 'PD СКБ '+'ЮЛ СПАРК' #Application/Behavioral + CAR/CONSUMER/MORTGAGE/CARDS

ident = ['Название клиента',
         'ИНН',
         'Date']

## Целевая перменная - target, веса наблюдений - sample_weight

In [ ]:
target = 'DEF_WITHIN_1Y'
sample_weight = None

## Создание папок для работы

### Корневая папка

In [ ]:
path_in = r'C:\Users\egorenkors\Desktop\IRB_ЮЛ\PD\in' #Папка с входящими данными
path_out = r'C:\Users\egorenkors\Desktop\IRB_ЮЛ\PD\out' #Папка в которой будет результат работы скрипта

tt_in = os.path.join(path_in,'*') # Папка с обучающей выборкой

# Создание папки вывода
path_out = os.path.join(path_out, f'{reg_name}')
directories = [path_out]

for dir in directories:
    try:
        os.makedirs(dir)    
        print("Directory " , dir ,  " Created ")
    except FileExistsError:
        print("Directory " , dir ,  " already exists")

### Вложенные папки

In [ ]:
directories = [os.path.join(path_out,
                            r'WoE_графики'),
               os.path.join(path_out,
                            r'PSI_графики'),
               os.path.join(path_out,
                            r'Использованные_данные'),
               os.path.join(path_out,
                            r'Преобразованные_данные'),              
               os.path.join(path_out,
                            r'Данные_итог'),      
               os.path.join(path_out,
                            r'Компоненты_модели_pickle'),
               os.path.join(path_out,
                            r'Компоненты_модели_pickle\Биннинг'),
               os.path.join(path_out,
                            r'Компоненты_модели_pickle\Компоненты_ядра'),
               os.path.join(path_out,
                            r'Графики распределений')
              ]
for dir in directories:
    try:
        os.makedirs(dir)    
        print("Directory " , dir ,  " Created ")
    except FileExistsError:
        print("Directory " , dir ,  " already exists")

# ПЕРВЫЙ ВВОД ДАННЫХ

## Ввод данных train and test

### train and test (tt)

In [ ]:
file_list = glob.glob(tt_in)
tt = pd.DataFrame()
for f in file_list:
    data = pd.read_excel(f)
    tt = pd.concat([tt, data])
    tt= tt.reset_index(drop = True)
    del data

f_name = os.path.basename(f)    
to_print = f_name+' loaded ' + str(tt.shape[0]) + ' obs'

# Сохранение исходника тренировочной выборки
save_file = os.path.join(path_out, r'Использованные_данные', f_name)
tt.to_csv(save_file, index=False)
print(to_print)

### Разделение показателей на категориальные и числовые

In [ ]:
all_feats = tt.drop(columns=ident+[target,
                                      sample_weight], 
                       errors = 'ignore').columns.to_list()
numerical = list(tt[all_feats]._get_numeric_data().columns)
dummies = list(tt[numerical].loc[:, tt[numerical].nunique()<=2].columns)
not_dummies = [col for col in tt[dummies].columns if 'num' in col]
dummies = list(set(dummies)-set(not_dummies))
numerical = list(set(numerical)-set(dummies))
categorical = list(set(all_feats) - set(numerical))
not_cats = [col for col in tt[categorical].columns if 'num' in col]
not_cats_2 = [col for col in tt[categorical].columns if 'std' in col]
categorical = list(set(categorical) - set(not_cats) - set(not_cats_2))
numerical = list(set(all_feats) - set(categorical))

#### Обзор для внесения правок

##### Числовые

In [ ]:
# tt[numerical].head(5)

##### Категориальные

In [ ]:
# tt[categorical].head(5)

##### Правка

In [ ]:
# do something

## Обработка +/- бесонечности

In [ ]:
for x in numerical:
    inf_replacer(tt,
                 x,
                 n_min_max = 3)

## Описательные статистики

### Описательные статистики для количественных показателей

In [ ]:
num_descr = tt[numerical].describe(percentiles=[.05, .25, .5, .75, .95]).transpose().round(3)
num_descr['nan, %'] = ((tt[numerical].isna().sum()/tt[numerical].shape[0])).round(3)
num_descr = num_descr.drop(['count'], axis=1, errors='ignore')
num_descr = num_descr
num_descr = num_descr.rename(columns ={'50%':'50%(median)', 'index':'feature'}, errors='ignore')

save_file = os.path.join(path_out, 'Desc numerical feats.xlsx')
num_descr.to_excel(save_file)
# num_descr

### Описательные статистики для качественных показателей

In [ ]:
cat_descr = tt[categorical].astype('object').describe(include='object').transpose()
cat_descr['nan, %'] = ((tt[categorical].isna().sum()/tt[categorical].shape[0])).round(3)
cat_descr = cat_descr.drop(['count','freq'], axis=1, errors='ignore')
cat_descr = cat_descr
cat_descr = cat_descr.rename(columns ={'top':'mode', 'index':'feature', 'unique':'n_unique'}, errors='ignore')

save_file = os.path.join(path_out, 'Desc categorical feats.xlsx')
cat_descr.to_excel(save_file)
# cat_descr

## Первичный контроль качества данных и трансформация

### Фильтр по высокому количеству пропусков (<90%)

In [ ]:
num_na_filter = list(num_descr[num_descr['nan, %']>0.9].index)
numerical = list(set(numerical)-set(num_na_filter))
cat_na_filter = list(cat_descr[cat_descr['nan, %']>0.9].index)
categorical = list(set(categorical) - set(cat_na_filter))

na_filter = num_na_filter+cat_na_filter

### Менее 2-х уникальных значений

In [ ]:
nunique_less_2 = tt[all_feats].columns[tt[all_feats].nunique()<2].to_list()
numerical = list(set(numerical)-set(nunique_less_2))
categorical = list(set(categorical) - set(nunique_less_2))

### Показатели с высокой концентраией >95%

In [ ]:
high_concentrated = []
for x in all_feats:
    if tt[x].value_counts().max()/tt.shape[0] > 0.95:
        high_concentrated = high_concentrated + [x]
    else:
        pass
numerical = list(set(numerical)-set(high_concentrated))
categorical = list(set(categorical) - set(high_concentrated))

### Данные для трансформации

#### train - test split

In [ ]:
test = tt.sample(frac= 0.2, 
                 random_state=seed)
train = tt[~tt.index.isin(test.index)]
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)
del tt
gc.collect()

### Трансформация (WoE-биннинг) показателей

In [ ]:
binning_failed = []

In [ ]:
from tqdm import tqdm

In [ ]:
warnings.simplefilter('ignore', UserWarning)
# warnings.simplefilter('default', UserWarning)
np.set_printoptions(suppress=False)
np.set_printoptions(formatter={'all':lambda x: str(x)})

#### Числовые показатели

##### Автоматический биннинг 

In [ ]:
for x in tqdm(numerical):
    try:
        bin_model_file = os.path.join(path_out, r'Компоненты_модели_pickle\Биннинг','bin_model_'+str(x)+'.sav')
        locals()['bin_model_' + str(x)] = pickle.load(open(bin_model_file, 'rb'))
    except FileNotFoundError:
        try:
            locals()['bin_model_' + str(x)] = numerical_binning(target_type = 'binary',
                                                            n_bins = 5,
                                                            min_bin_size = 0.05,
                                                            outlier_detector = 'knn',
                                                            seed = seed).fit(df = train,
                                                                             feature = x,
                                                                             target = target).calculate_woe()
            save_file = os.path.join(path_out, r'Компоненты_модели_pickle\Биннинг','bin_model_'+str(x)+'.sav')
            pickle.dump(locals()['bin_model_' + str(x)], 
                        open(save_file, 'wb'))
        except:
            binning_failed = binning_failed + [x]
            pass
    if x in binning_failed:
        pass
    else:
        locals()['woe_' + str(x)] = locals()['bin_model_' + str(x)].get_woe_table()

##### Внесение правок

###### x...

In [ ]:
# do_something like bin_model_feature.user_split_points(...).calculate_woe() ... _.get_woe_table()

#### Категориальные показатели

##### Автоматический биннинг 

In [ ]:
for x in tqdm(categorical):
    try:
        bin_model_file = os.path.join(path_out, r'Компоненты_модели_pickle','bin_model_'+str(x)+'.sav')
        locals()['bin_model_' + str(x)] = pickle.load(open(bin_model_file, 'rb'))
    except FileNotFoundError:
        try:
            locals()['bin_model_' + str(x)] = categorical_binning(target_type = 'binary',
                                                                  n_bins = 4,
                                                                  min_bin_size = 0.05,
                                                                  cat_other_cutoff = 0.01,
                                                                  seed = seed).fit(df=train,
                                                                                   feature = x,
                                                                                   target = target).calculate_woe()
            save_file = os.path.join(path_out, r'Компоненты_модели_pickle\Биннинг','bin_model_'+str(x)+'.sav')
            pickle.dump(locals()['bin_model_' + str(x)], 
                        open(save_file, 'wb'))
        except:
            binning_failed = binning_failed + [x]
            pass
    if x in binning_failed:
        pass
    else:
        locals()['woe_' + str(x)] = locals()['bin_model_' + str(x)].get_woe_table()

#### Внесение правок

In [ ]:
# do_something like bin_model_feature.user_splits(user_splits = {1:('a','b','c'), 2:('d','e','f'),...,'Special':('...'),...})
# _.calculate_woe() ... _.get_woe_table()

### Итоговые таблицы биннинга

In [ ]:
numerical = list(set(numerical) - set(binning_failed))
categorical = list(set(categorical) - set(binning_failed))
adj_feats =  numerical + categorical

#### Биннинг числовых показателей

In [ ]:
num_woe_table_total = pd.DataFrame()
for x in numerical:
    num_woe_table_total = pd.concat([num_woe_table_total,
                                    locals()['woe_' + str(x)].reset_index()], 
                                    ignore_index = True)
save_file = os.path.join(path_out, 'Binning numerical feats.xlsx')
num_woe_table_total.to_excel(save_file, 
                             index=False)

# num_woe_table_total

#### Биннинг категориальных показателей

In [ ]:
cat_woe_table_total = pd.DataFrame()
for x in categorical:
    cat_woe_table_total = pd.concat([cat_woe_table_total,
                                    locals()['woe_' + str(x)].reset_index()], 
                                    ignore_index = True)
save_file = os.path.join(path_out, 'Binning categorical feats.xlsx')
cat_woe_table_total.to_excel(save_file, 
                             index=False)

# cat_woe_table_total

### Результат первичной обработки показателей

In [ ]:
adj_feats = numerical + categorical

adj_res = pd.DataFrame({'Показатели': ['Исходной набор',
                                       'Отсеиваемые при проверке доли пропусков (>0.9)', 
                                       'Отсеиваемые при проверке количества уникальных значений (<2)', 
                                       'Отсеиваемые при проверке конецентрации наблюдений на значениях (>0.95)',
                                       'Отсеиваемые из-за невозможности применения алгоритма трансформации',
                                       'Очищенный набор'],
                       'Список показателей': [all_feats, 
                                              na_filter,
                                              nunique_less_2,
                                              high_concentrated,
                                              binning_failed,
                                              adj_feats],
                       'Кол-во показателей в списке':[len(all_feats),
                                                      len(na_filter),
                                                      len(nunique_less_2),
                                                      len(high_concentrated),
                                                      len(binning_failed),
                                                      len(adj_feats)]})

adj_res = adj_res.set_index('Показатели')
save_file = os.path.join(path_out, 'Результаты первичного анализа качества '+f'{reg_name}'+'.xlsx')
adj_res.to_excel(save_file)

adj_res

# Первый вывод данных

## Трансформация показателей в выборках

In [ ]:
for x in tqdm(adj_feats):
    train = locals()['bin_model_' + str(x)].woe_transform(df=train)
    test = locals()['bin_model_' + str(x)].woe_transform(df=test)

#### Обработка пропусков и новых значений

In [ ]:
woe_feats = list(train.columns[train.columns.str.contains(r'^WoE')])
bin_indexs = list(train.columns[train.columns.str.contains(r'^bin_index')])


for x in woe_feats:
    test[x] = test[x].fillna(0)
for x in bin_indexs:
    test[x] = test[x].fillna('Missing')

#### Сохранение преобразованных выборок

In [ ]:
transformed_cols = list(train.columns[train.columns.str.contains(r'^WoE')])+\
list(train.columns[train.columns.str.contains(r'^bin')])

if sample_weight is not None:
    cols = ident+[target,sample_weight]+transformed_cols
else:
    cols = ident+[target]+transformed_cols

df_list = ['train',
           'test']

for f in df_list:
    f_name = f+'_transformed'
    save_file = os.path.join(path_out, r'Преобразованные_данные',
                             f_name+'.csv')
    locals()[f][cols].to_csv(save_file, index=False)
    del locals()[f]

# Второй ввод данных

In [ ]:
df_list = ['train',
           'test']

for f in df_list:
    f_name = f+'_transformed'
    file = os.path.join(path_out+r'\Преобразованные_данные',f_name+'.csv')
    data = pd.read_csv(file, 
                       low_memory=False)
    locals()[f] = pd.DataFrame()
    locals()[f] = pd.concat([locals()[f], 
                             data])
    locals()[f] = locals()[f].reset_index(drop=True)
    del data

woe_feats = list(train.columns[train.columns.str.contains(r'^WoE')])
bin_indexs = list(train.columns[train.columns.str.contains(r'^bin_index')])

## Однофакторный анализ

### Information Value

In [ ]:
IVs = pd.DataFrame()
for x in adj_feats:
    IVs = pd.concat([IVs,
                     locals()['woe_' + str(x)][['feature','IV']].reset_index(drop=True).iloc[:1]], 
                    ignore_index=True).reset_index(drop=True)
IVs['IV'] = IVs['IV'].fillna(0)
IVs = IVs.set_index('feature').sort_values(by='IV')
IVs = IVs.sort_values(by='IV', ascending=False)
save_file = os.path.join(path_out, 'IVs '+f'{reg_name}'+'.xlsx')
IVs.to_excel(save_file)

iv_below_002 = list('WoE_' + x for x in list(IVs.loc[round(IVs['IV'],2) < 0.02].index))
iv_above_05 = list('WoE_' + x for x in list(IVs.loc[round(IVs['IV'],2) > 0.5].index))


IVs.head(20)

### Логическая проверка результатов биннинга

#### Количественные показатели

In [ ]:
# do something

#### Качественные показатели

In [ ]:
# do something

#### Итоговый список

In [ ]:
logical_check_fail = []

### Анализ Gini однофакторных моделей

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, log_loss, classification_report, confusion_matrix, roc_curve, auc

estimator = LogisticRegression(solver='liblinear', random_state=seed)

y_train = train[target].values.ravel()
y_test = test[target].values.ravel()

Gini_UA = pd.DataFrame(columns = ['Feature', 
                                  'Gini train', 
                                  'Gini test'])

for x in woe_feats:
    x_train = train[x].values.reshape(-1,1)
    x_test = test[x].values.reshape(-1,1)
    
    model = estimator.fit(x_train, y_train)  
  
    gini_train = 2*roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1
    gini_test = 2*roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1
    
    new_row = pd.DataFrame([{'Feature': x, 
                             'Gini train' : gini_train,
                             'Gini test' : gini_test}])
    Gini_UA = pd.concat([Gini_UA,
                         new_row], ignore_index=True)
Gini_UA['Gini test-train rel_deviation'] = abs(Gini_UA['Gini test']/Gini_UA['Gini train']-1)
Gini_UA = Gini_UA.set_index('Feature')

Gini_UA = Gini_UA.sort_values(by='Gini test',
                              ascending = False)

gini_below_005 = list(Gini_UA.loc[round(Gini_UA['Gini train'],2) < 0.05].index)
gini_rel_dev_above_025 = list(Gini_UA.loc[round(Gini_UA['Gini test-train rel_deviation'], 2) >= 0.25].index)

save_file = os.path.join(path_out, '1D Gini '+f'{reg_name}'+'.xlsx')
Gini_UA.to_excel(save_file)

Gini_UA.head(20)

### PSI (индекс стабильности популяции)

In [ ]:
PSI_table = pd.DataFrame(columns = ['Feature', 
                                    'PSI'])
PSI_table_detailed = pd.DataFrame()

for x in adj_feats:
    x_psi = psi_().fit(dev = train,
                       val = test,
                       feature = x)
    save_file = os.path.join(path_out, 
                             r'PSI_графики')
    x_psi.plot(save = save_file)
    new_row = pd.DataFrame([{'Feature': 'WoE_'+x, 
                             'PSI':x_psi.get_psi()}])
    PSI_table = pd.concat([PSI_table,
                           new_row], ignore_index=True)
    PSI_table_detailed = pd.concat([PSI_table_detailed,
                                    x_psi.get_table().reset_index()],
                                   ignore_index=True)



PSI_table = PSI_table.set_index('Feature')
psi_above_025 = list(PSI_table.loc[round(PSI_table['PSI'], 2) >= 0.25].index)

PSI_table = PSI_table.sort_values(by='PSI',
                                 ascending = False)

save_file = os.path.join(path_out, 'PSI '+f'{reg_name}'+'.xlsx')
PSI_table.to_excel(save_file)

PSI_table_detailed = PSI_table_detailed.set_index('Feature')
save_file = os.path.join(path_out, 'PSI detailed '+f'{reg_name}'+'.xlsx')
PSI_table_detailed.to_excel(save_file)

# PSI_table.head(20)

### Spearman r - коэфф. корреляции Спирмена для проверки стабильности (<0.3)

In [ ]:
sprmn_stab = pd.DataFrame(columns= ['feature',
                                    'Spearman_r_(dev/test)',
                                    'clean_Spearman_r_(dev/test)']
                         )
for x in adj_feats:
    try:
        part = spearmanr_stability(dev = train,
                                   val = test,
                                   feature = x,
                                   target = target)
        part['feature'] = x
        sprmn_stab = pd.concat([sprmn_stab,
                                part], ignore_index=True)
    except ValueError:
        part['feature'] = x
        part['Spearman_r_(dev/test)'] = np.nan
        part['clean_Spearman_r_(dev/test)'] = np.nan
        sprmn_stab = pd.concat([sprmn_stab,
                                part], ignore_index=True)

sprmn_stab = sprmn_stab.fillna(0)
sprmn_stab = sprmn_stab.set_index('feature')

sprmn_stab = sprmn_stab.sort_values(by='clean_Spearman_r_(dev/test)')

save_file = os.path.join(path_out, 'Spearman_R '+f'{reg_name}'+'.xlsx')
sprmn_stab.to_excel(save_file)

sprmnr_below_03 = list(sprmn_stab.loc[round(sprmn_stab['Spearman_r_(dev/test)'],2) < 0.3].index)

# sprmn_stab.head(20)

### Результат однофакторного анализа

#### Решение о (не-)применении критериев фильтрации однофакторного анализа ('y'/'n')

In [ ]:
# Фильтр по IV "снизу"
iv_low_filter = 'y' 

# Фильтр по IV "сверху"
iv_top_filter = 'n'

# Фильтр по Gini "снизу"
gini_low_filter = 'y'

# Фильтр по изменение Gini в test
gini_dev_filter = 'n'

# Фильтр по логической непротиворечивости
logical_filter = 'n'

# Фильтр по PSI (стабильность)
psi_filter = 'n'

# Фильтр по Spearman R (стабильность)
spearman_r_filter = 'n'

#### Таблица с результатами 1D анализа

In [ ]:
woe_adj = woe_feats.copy()

## IV
### Фильтр - IV ниже 0.02
if iv_low_filter == 'y':
    woe_adj = list(set(woe_adj)-set(iv_below_002))
else:
    pass
### Фильтр - IV выше 0.5
if iv_top_filter == 'y':
    woe_adj = list(set(woe_adj)-set(iv_above_05))
else:
    pass

## Gini
### Фильтр - Gini ниже 0.05
if gini_low_filter == 'y':
    woe_adj = list(set(woe_adj)-set(gini_below_005))
else:
    pass
### Фильтр - Относительное изменение Gini >25%
if gini_dev_filter == 'y':
    woe_adj = list(set(woe_adj)-set(gini_rel_dev_above_025))
else:
    pass

## PSI
### PSI выше 0.25
if psi_filter == 'y':
    woe_adj = list(set(woe_adj)-set(psi_above_025))
else:
    pass

## Spearman_r
### Spearman_r < 0.3
if spearman_r_filter == 'y':
    woe_adj = list(set(woe_adj)-set(sprmnr_below_03))
else:
    pass

## Бизнес-логика
### Фильтр - соотвествие показателя бизнес-логике
if logical_filter == 'y':
    woe_adj = list(set(woe_adj)-set(logical_check_fail))
else:
    pass


adj_res = pd.DataFrame({'Показатели': ['Исходной набор',
                                       'Отсеиваемые при проверке значений IV (<0.02)', 
                                       'Отсеиваемые при проверке значений IV (>0.5)', 
                                       'Отсеиваемые при проверке значений Gini (<0.05)',
                                       'Отсеиваемые при проверки стабильности Gini (rel_dev<0.25)',
                                       'Отсеиваемые при проверке значений PSI (>0.25)',
                                       'Отсеиваемые при проверке значений SpearmanR (<0.3)',
                                       'Отсеиваемые при логической проверке', 
                                       'Очищенный набор'],
                       'Список показателей': [woe_feats, 
                                              iv_below_002,
                                              iv_above_05,
                                              gini_below_005,
                                              gini_rel_dev_above_025, 
                                              psi_above_025,
                                              sprmnr_below_03,
                                              logical_check_fail,
                                              woe_adj],
                       'Кол-во показателей в списке':[len(woe_feats),
                                                      len(iv_below_002),
                                                      len(iv_above_05),
                                                      len(gini_below_005),
                                                      len(gini_rel_dev_above_025),
                                                      len(psi_above_025),
                                                      len(sprmnr_below_03),
                                                      len(logical_check_fail),
                                                      len(woe_adj)],
                       'Использование фильтра y/n':['-',
                                                    iv_low_filter,
                                                    iv_top_filter,
                                                    gini_low_filter,
                                                    gini_dev_filter,
                                                    psi_filter,
                                                    spearman_r_filter,
                                                    logical_filter,
                                                   '-']})

adj_res = adj_res.set_index('Показатели')
save_file = os.path.join(path_out, 'Результаты 1D анализа '+f'{reg_name}'+'.xlsx')
adj_res.to_excel(save_file)

adj_res

# Второй вывод данных

In [ ]:
if sample_weight is not None:
    cols = ident+[target,sample_weight]+woe_adj
else:
    cols = ident+[target]+woe_adj

In [ ]:
df_list = ['train',
           'test']

for f in df_list:
    f_name = f+'_transformed_adjusted'
    save_file = os.path.join(path_out, r'Преобразованные_данные',
                             f_name+'.csv')
    locals()[f][cols].to_csv(save_file, index=False)
    del locals()[f]

# Третий ввод данных

In [ ]:
warnings.simplefilter(action='ignore', 
                      category=pd.errors.SettingWithCopyWarning)

#### Выборки

In [ ]:
df_list = ['train',
           'test']

for f in df_list:
    f_name = f+'_transformed_adjusted'
    file = os.path.join(path_out+r'\Преобразованные_данные',f_name+'.csv')
    data = pd.read_csv(file, 
                       low_memory=False)
    locals()[f] = pd.DataFrame()
    locals()[f] = pd.concat([locals()[f], 
                             data])
    locals()[f] = locals()[f].reset_index(drop=True)
    del data

woe_feats = list(train.columns[train.columns.str.contains(r'^WoE')])

## Многофакторный анализ

In [ ]:
try:
    overfitting_feats
except NameError:
    overfitting_feats = []
else:
    print(overfitting_feats)

In [ ]:
try:
    high_vif_feats
except NameError:
    high_vif_feats = []
else:
    print(high_vif_feats)

In [ ]:
woe_feats = list(set(woe_feats)-set(overfitting_feats)-set(high_vif_feats))

## Отбор показателей

In [ ]:
# from sklearn.linear_model import LogisticRegressionCV

### Параметры модели (Логит)

In [ ]:
from sklearn.linear_model import LogisticRegression

# penalty 'L1', 'l2', 'elasticnet'
# elacticnet = l1&l2

estimator = LogisticRegression(solver='saga',
                               max_iter=100,
                               penalty = 'elasticnet',
                               C=1.0, 
                               fit_intercept=True,
                               random_state = seed,
                               n_jobs = -1,
                               l1_ratio=0.5)

# Не вижу смысла в подборе гипер-параметров, т.к. это оказывает минимальное влияние на линейные регрессионные модели
# Попробовать по-экспериментировать с l2 и elasticnet и мультиколлениарностью 

### Отбор показателей (mlxtend SFFS)

In [ ]:
# Поробовать backawrd

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import StratifiedKFold

#### Алгоритм SFS (сейчас floating forward)

In [ ]:
cv = StratifiedKFold(n_splits=5, 
                     shuffle = True, 
                     random_state=seed)

# По умолчанию выбирает 10 лучших, финальный размер набора показателей определяется на усмотрение моделиста
sfs = SFS(estimator = estimator, 
          k_features=10, 
          forward=True, 
          floating=True, 
          verbose = 10,
          scoring='roc_auc',
          n_jobs = 1,
          cv=cv)

sfs = sfs.fit(X = train[woe_feats],
              y = train[target])

#### Таблица результатов

In [ ]:
table_sfs = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
table_sfs = table_sfs.drop(['cv_scores', 'feature_idx'], axis = 1)
table_sfs = table_sfs.rename(columns={'feature_names':'selected features','avg_score': 'avg_auc_roc'})

table_sfs['avg_auc_roc increase']=np.nan
for n in list(range(2, table_sfs.shape[0])):
    table_sfs['avg_auc_roc increase'][n]=(table_sfs['avg_auc_roc'][n]-table_sfs['avg_auc_roc'][n-1]).round(3)
table_sfs['avg_auc_roc increase']=np.where(table_sfs['avg_auc_roc increase'].isna(), 
                                        table_sfs['avg_auc_roc'], 
                                        table_sfs['avg_auc_roc increase'])

table_sfs = table_sfs[['selected features',
                    'avg_auc_roc', 
                    'avg_auc_roc increase', 
                    'ci_bound', 
                    'std_dev',
                    'std_err']]
table_sfs.index = table_sfs.index.rename('features n')

save_file = os.path.join(path_out, 'SFS (выбор показателей) '+f'{reg_name}'+'.xlsx')
table_sfs.to_excel(save_file)

table_sfs

#### Указать число включаемых в модель показателей n_features

In [ ]:
n_features = 7

best_feats = list(table_sfs['selected features'][n_features])
best_feats

### Тест на переобученность

In [ ]:
x_train = train[best_feats]
y_train = train[target]

x_test = test[best_feats]
y_test = test[target]


#Обучение модели
model = estimator.fit(x_train, 
                      y_train)

over_test_table = pd.DataFrame(columns = ['Gini train', 
                               'Gini test', 
                               'Исключаемый фактор',
                               'Падение Gini на обучающей выборке',
                               'Падение Gini на тестовой выборке',
                               'overfitting'])

gini_test = 2*roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1
gini_train = 2*roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1

new_row = {'Gini train':gini_train,
           'Gini test':gini_test,
           'Исключаемый фактор': 'Все включены'}
over_test_table = pd.concat([over_test_table, pd.DataFrame([new_row])],ignore_index=True)

for x in best_feats:
    x_subset = best_feats.copy()
    x_subset.remove(x)    
    x_train_1 = train[x_subset]
    y_train_1 = train[target]
    x_test_1 = test[x_subset]
    y_test_1 = test[target]
    m_test = estimator.fit(x_train_1,  y_train_1)
  
    gini_test = 2*roc_auc_score(y_test_1, 
                                m_test.predict_proba(x_test_1)[:,1]) - 1
    
    gini_train = 2*roc_auc_score(y_train_1, 
                                 m_test.predict_proba(x_train_1)[:,1]) - 1
    new_row = {'Gini train':gini_train.round(3),'Gini test':gini_test.round(3), 'Исключаемый фактор': x}
    over_test_table = pd.concat([over_test_table, pd.DataFrame([new_row])],ignore_index=True)

rows = list(range(1, over_test_table.shape[0]))
for n in rows:
    over_test_table['Падение Gini на обучающей выборке'][n] = over_test_table['Gini train'][0]-over_test_table['Gini train'][n]
    over_test_table['Падение Gini на тестовой выборке'][n] = over_test_table['Gini test'][0]-over_test_table['Gini test'][n]
    over_test_table['overfitting'][n] = np.where((over_test_table['Падение Gini на обучающей выборке'][n]>0.1), 1, 0)
over_test_table = over_test_table.set_index('Исключаемый фактор')


save_file = os.path.join(path_out, 'overfitting  test'+f'{reg_name}'+'.xlsx')
over_test_table.to_excel(save_file)    
over_test_table

#### Список показателей на которые модель переобучается

In [ ]:
overfitting_feats = []

### Проверка стат.-значимости переменных (p-значения)

In [ ]:
model = estimator.fit(x_train,
                      y_train)
params = np.append(model.intercept_,model.coef_)
predictions = model.predict(x_train)
newX = pd.DataFrame({'Constant':np.ones(len(x_train))}).join(pd.DataFrame(x_train))
MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))
var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/sd_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)))) for i in ts_b]
sd_b = np.round(sd_b,9)
ts_b = np.round(ts_b,9)
p_values = np.round(p_values,9)
params = np.round(params,9)
coeffs = pd.DataFrame()
coeffs['Coefficient'],coeffs['Std. err'],coeffs['p-values'] = [params,sd_b,p_values]
coeffs['Feature']=['intercept']+list(model.feature_names_in_)
coeffs = coeffs.set_index('Feature')
save_file = os.path.join(path_out, 'Модель '+f'{reg_name}'+'.xlsx')
coeffs.to_excel(save_file)
gc.collect()
coeffs

#### Список незначимых показателей

In [ ]:
high_p_value_feats = []

### Variance inflation factor (VIF) для проверки мультиколлинеарности

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Если значение больше 5 тогда следует исключить фактор

In [ ]:
X = train[best_feats]
X['intercept'] = coeffs['Coefficient'][0]
X = X[['intercept']+best_feats]
VIF = pd.DataFrame()
VIF['VIF'] = [variance_inflation_factor(X.values, i) 
              for i in range(X.shape[1])]
VIF['Feature'] = X.columns 
VIF = VIF.set_index('Feature')
save_file = os.path.join(path_out, 'VIF test' + f' {reg_name}'+'.xlsx')
VIF.to_excel(save_file)
VIF

In [ ]:
high_vif_feats = []

## Метрики качества полученной модели и их визуализация

### ROC-кривая

In [ ]:
import pylab as pl

In [ ]:
x_train = train[best_feats]
y_train = train[target].values

x_test = test[best_feats]
y_test = test[target].values

model = estimator.fit(x_train, 
                      y_train)

# График ROC

plt.figure(figsize=(10,8))

probas_train = model.predict_proba(x_train) # еще раз расчет вероятностей
fpr, tpr, thresholds = roc_curve(y_train, probas_train[:, 1])
roc_auc  = auc(fpr, tpr) # формула расчета ROC
pl.plot(fpr, tpr, label='%s AUC = %0.2f'  % ('train:',roc_auc) +';' + '%s Gini = %0.2f ' % ('', round(2*roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1, 3))) 
pl.plot([0, 1], [0, 1], 'k--')
pl.legend(loc=0)

probas_test = model.predict_proba(x_test) # еще раз расчет вероятностей
fpr, tpr, thresholds = roc_curve(y_test, probas_test[:, 1])
roc_auc  = auc(fpr, tpr) # формула расчета ROC
pl.plot(fpr, tpr, label='%s AUC = %0.2f'  % ('test:',roc_auc) +';' + '%s Gini = %0.2f ' % ('', round(2*roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1, 3)))
pl.plot([0, 1], [0, 1], 'k--')
pl.legend(loc=0)


plt.grid()
NAME = 'Logit '+f'{reg_name}'
plt.title(NAME)
save_file = os.path.join(path_out, 'ROC_curve '+f'{reg_name}'+'.png')
plt.savefig(save_file, 
            bbox_inches='tight', 
            pad_inches=0.05, 
            dpi=360)

# Выгрузка pickle

### Модель logit

In [ ]:
save_file = os.path.join(path_out, 
                         r'Компоненты_модели_pickle\Компоненты_ядра','Model '+f'{reg_name}'+'.sav')
pickle.dump(model, 
            open(save_file, 'wb'))

### Биннинг показателей в модели

In [ ]:
for x in best_feats:
    feat = str(x).replace('WoE_', '') 
    save_file = os.path.join(path_out, 
                         r'Компоненты_модели_pickle\Компоненты_ядра','bin_model_'+f'{feat}'+'.sav')
    pickle.dump(locals()['bin_model_' + str(feat)], 
                open(save_file, 'wb'))

In [ ]:
reg_name